In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q -U google-generativeai pillow pdf2image opencv-python # Añadido opencv-python
!apt-get install -y poppler-utils # Necesario para pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import google.generativeai as genai
from PIL import Image
import os
from pdf2image import convert_from_path
import json

# --- Configuración de la API Key ---
from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key cargada y configurada exitosamente.")
except userdata.SecretNotFoundError:
    print("Error: El secreto 'GOOGLE_API_KEY' no fue encontrado.")
    print("Por favor, añade tu API Key a los secretos de Colab.")
    GOOGLE_API_KEY = None # Asegurarse de que GOOGLE_API_KEY es None si no se encuentra
except Exception as e:
    print(f"Ocurrió un error al configurar la API Key: {e}")
    GOOGLE_API_KEY = None # Asegurarse de que GOOGLE_API_KEY es None si hay error

# --- Configuración del Modelo ---
# (Asegúrate de que GOOGLE_API_KEY se haya cargado correctamente antes de esto)
if GOOGLE_API_KEY:
    model_name = 'gemini-1.5-flash-latest' # o 'gemini-1.5-pro-latest'
    model = genai.GenerativeModel(model_name)
    print(f"Modelo {model_name} inicializado.")
else:
    model = None
    print("El modelo no se pudo inicializar porque la API Key no está configurada.")

API Key cargada y configurada exitosamente.
Modelo gemini-1.5-flash-latest inicializado.


In [ ]:
# import os
# import shutil

# # Crear directorios si no existen
# IMAGES_DIR = "pdf_images"
# if os.path.exists(IMAGES_DIR):
#     shutil.rmtree(IMAGES_DIR)
# os.makedirs(IMAGES_DIR, exist_ok=True)

# pdf_path = '/content/drive/MyDrive/DEUDA_PREVISIONAL/PDFS/2017.pdf' # Asegúrate que esta ruta sea correcta

# if not os.path.exists(pdf_path):
#     print(f"Error: No se encontró el archivo PDF en la ruta especificada: {pdf_path}")
#     image_paths = []
# else:
#     print(f"Archivo '{pdf_path}' encontrado.")

#     try:
#         print("Convirtiendo PDF a imágenes...")
#         from pdf2image import convert_from_path
#         images = convert_from_path(pdf_path, dpi=300)
#         image_paths = []
#         for i, image in enumerate(images):
#             image_path = os.path.join(IMAGES_DIR, f"page_{i+1}.png")
#             image.save(image_path, "PNG")
#             image_paths.append(image_path)
#             print(f"Página {i+1} guardada como {image_path}")
#         print(f"Conversión completa. {len(image_paths)} imágenes generadas en la carpeta '{IMAGES_DIR}'.")
#     except Exception as e:
#         print(f"Error al convertir el PDF a imágenes: {e}")
#         image_paths = []

In [ ]:
# Especifica la ruta de la carpeta en Google Drive que contiene las imágenes
drive_image_folder_path = '/content/drive/MyDrive/DEUDA_PREVISIONAL/PDFS/2017'

# Verifica si la carpeta existe
if not os.path.exists(drive_image_folder_path):
    print(f"Error: No se encontró la carpeta de imágenes en la ruta especificada: {drive_image_folder_path}")
    image_paths = []
else:
    print(f"Carpeta '{drive_image_folder_path}' encontrada.")

    # Lista todos los archivos dentro de la carpeta y filtra solo los archivos de imagen (puedes ajustar las extensiones)
    all_files_in_folder = os.listdir(drive_image_folder_path)
    image_extensions = ('.png') # Define las extensiones de imagen que te interesan
    image_paths = [os.path.join(drive_image_folder_path, filename)
                   for filename in all_files_in_folder
                   if filename.lower().endswith(image_extensions) and os.path.isfile(os.path.join(drive_image_folder_path, filename))]

    if not image_paths:
        print("No se encontraron archivos de imagen en la carpeta especificada.")
    else:
        print(f"Se encontraron {len(image_paths)} archivos de imagen en la carpeta.")

Carpeta '/content/drive/MyDrive/DEUDA_PREVISIONAL/PDFS/2017' encontrada.
Se encontraron 31 archivos de imagen en la carpeta.


In [ ]:
# CELDA QUE CONTIENE extract_data_from_image_gemini_v_few_shot (MODIFICADA PARA PREPROCESAMIENTO)
import google.generativeai as genai
from PIL import Image
import json
import os

# (Asegúrate de que 'model' esté definido globalmente o se pase como argumento)

def extract_data_from_image_gemini_v_few_shot_pil(pil_image_obj, target_dia_pago="31.12.2017", image_identifier="PIL_Image_Object"):
    if 'model' not in globals() or globals()['model'] is None:
        print("Error: La variable global 'model' de Gemini no está inicializada.")
        print("Asegúrate de haber configurado la API Key y el modelo en una celda anterior.")
        return None

    if not isinstance(pil_image_obj, Image.Image):
        print(f"Error: El objeto proporcionado no es una imagen PIL.Image válida para {image_identifier}")
        return None

    img_to_send = pil_image_obj

    prompt = f"""
    Analiza la siguiente imagen, que contiene tablas de datos financieros. Tu tarea es extraer información específica de la fila donde "DIA DE PAGO" es "{target_dia_pago}".

    La imagen presenta datos financieros organizados en columnas, donde cada columna principal representa un mes y año (ej. "MAYO 1997", "JUNIO 1997", etc.). Debes procesar estas columnas de mes/año de izquierda a derecha como las encuentres en la imagen.

    Para cada columna de mes/año que identifiques:
    1.  **mes_año_columna**: Extrae el nombre completo del mes y año que encabeza esta columna.
    2.  **reajuste_porcentaje**: Busca el valor de "REAJUSTE %" que está visualmente asociado y típicamente ubicado encima de la columna del mes/año actual. Extrae solo el valor porcentual, omitiendo la palabra "REAJUSTE" si está presente.
    3.  Localiza la fila de datos correspondiente a "{target_dia_pago}".
    4.  Dentro de esta fila "{target_dia_pago}" y asegurándote de que estás obteniendo los valores de la columna del mes/año actual que estás procesando:
        a.  **interes_afiliado_porcentaje**: Busca la sección principal de "INTERES" (usualmente "% INTERES"). Debajo de este encabezado, para el mes/año actual, extrae el valor de la primera columna de datos numéricos.
        b.  **recargo_beneficio_afiliado_porcentaje**: Busca la sección principal de "RECARGO BENEFICIO" (ej. "* RECARGO BENEFICIO %" o "% RECARGO BENEFICIO"). Esta sección tiene subcolumnas "AFILIADO" y "A.F.P.". Para el mes/año actual, extrae el valor específicamente de la subcolumna "AFILIADO". Ignora la subcolumna "A.F.P.". Si la celda "AFILIADO" bajo "RECARGO BENEFICIO" para el mes/año actual y la fecha objetivo está vacía o no es un valor numérico claro, asígnalo como null.

    **Es muy importante que intentes encontrar y extraer un valor para los tres campos (reajuste_porcentaje, interes_afiliado_porcentaje, recargo_beneficio_afiliado_porcentaje) para cada mes_año_columna que identifiques y que contenga datos en la fila objetivo. Asigna null solo si el valor está genuinamente ausente o es ilegible. No omitas campos si los datos correspondientes existen.**

    Presenta los resultados como una lista de objetos JSON, manteniendo el orden de izquierda a derecha en que aparecen los meses/años en la imagen. Cada objeto debe tener las claves: "mes_año_columna", "reajuste_porcentaje", "interes_afiliado_porcentaje", "recargo_beneficio_afiliado_porcentaje".

    Si para un mes/año particular no encuentras la fila "{target_dia_pago}" o los datos requeridos de forma clara, omite ese mes/año de la lista. Si no encuentras datos válidos para ningún mes/año en la imagen, devuelve una lista vacía.

    Aquí tienes ejemplos de la extracción correcta para la fila "{target_dia_pago}":
    Ejemplo 1:
    {{
      "mes_año_columna": "MAYO 1997",
      "reajuste_porcentaje": "13.70%",
      "interes_afiliado_porcentaje": "3293.80%",
      "recargo_beneficio_afiliado_porcentaje": "658.76%"
    }}
    Ejemplo 2:
    {{
      "mes_año_columna": "JUNIO 1997",
      "reajuste_porcentaje": "13.70%",
      "interes_afiliado_porcentaje": "3250.14%",
      "recargo_beneficio_afiliado_porcentaje": "650.03%"
    }}
    Ejemplo 3:
    {{
      "mes_año_columna": "JULIO 1997",
      "reajuste_porcentaje": "13.70%",
      "interes_afiliado_porcentaje": "3207.96%",
      "recargo_beneficio_afiliado_porcentaje": "641.59%"
    }}
    Ejemplo 4:
    {{
      "mes_año_columna": "AGOSTO 1997",
      "reajuste_porcentaje": "13.48%",
      "interes_afiliado_porcentaje": "3171.31%",
      "recargo_beneficio_afiliado_porcentaje": "634.38%"
    }}
    Ejemplo 5:
    {{
      "mes_año_columna": "SEPTIEMBRE 1997",
      "reajuste_porcentaje": "13.01%",
      "interes_afiliado_porcentaje": "3143.02%",
      "recargo_beneficio_afiliado_porcentaje": "628.60%"
    }}
    """
    print(f"\nProcesando imagen: {image_identifier} con Gemini, prompt v_few_shot (con énfasis anti-omisión)...")
    try:
        global_model = globals()['model']
        response = global_model.generate_content([prompt, img_to_send], stream=False)
        response.resolve()

        text_response = response.text
        print(f"Respuesta cruda de Gemini para {image_identifier}:\n{text_response}")

        json_str = text_response
        if "```json" in text_response:
            try:
                json_str = text_response.split("```json")[1].split("```")[0].strip()
            except IndexError:
                print(f"Advertencia: Se encontró '```json' pero no se pudo parsear el bloque para {image_identifier}.")
                json_str = text_response
        elif text_response.strip().startswith("```") and text_response.strip().endswith("```"):
            json_str = text_response.strip()[3:-3].strip()
        elif text_response.strip().startswith("[") and text_response.strip().endswith("]"):
             json_str = text_response.strip()
        else:
            print(f"Advertencia: La respuesta de Gemini para {image_identifier} no parece ser un bloque JSON estándar. Se intentará parsear directamente.")

        try:
            extracted_json = json.loads(json_str)
            return extracted_json
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON de la respuesta de Gemini para {image_identifier}: {e}")
            print(f"String que causó el error (json_str):\n>>>\n{json_str}\n<<<")
            print(f"Respuesta original completa (text_response):\n>>>\n{text_response}\n<<<")
            return None

    except Exception as e:
        print(f"Error durante la llamada a la API de Gemini para {image_identifier}: {e}")
        if hasattr(e, 'message'): # Para errores de la API de Google
            print(f"Detalle del error de API: {e.message}")
        elif hasattr(e, 'response') and hasattr(e.response, 'text'): # Para errores de requests (si usaras OpenRouter)
            print(f"Detalle del error de API (respuesta): {e.response.text}")
        return None

def extract_data_from_image_gemini_v_few_shot(image_path, target_dia_pago="31.12.2017"):
    try:
        img_pil = Image.open(image_path)
        return extract_data_from_image_gemini_v_few_shot_pil(img_pil, target_dia_pago, image_identifier=image_path)
    except FileNotFoundError:
        print(f"Error: No se encontró la imagen en la ruta {image_path}")
        return None
    except Exception as e:
        print(f"Error al abrir la imagen {image_path}: {e}")
        return None

In [ ]:
import os
from PIL import Image
import cv2 # <--- NUEVA IMPORTACIÓN
import numpy as np # <--- NUEVA IMPORTACIÓN

# Modifica la definición de la función para incluir los flags de preprocesamiento
def segmentar_y_procesar_pagina_v2(imagen_path_original, num_segmentos_verticales_por_tabla=2, overlap_px=50, aplicar_preprocesamiento=True,
                                   aplicar_upscaling=False, factor_upscaling=1.5,
                                   aplicar_escala_grises=True,
                                   aplicar_clahe=True, clip_limit_clahe=2.0, tile_grid_clahe=(8,8),
                                   aplicar_thresholding_adaptativo=True, block_size_thresh=15, c_value_thresh=5,
                                   aplicar_denoising_mediano=True, kernel_size_denoising=3,
                                   aplicar_nitidez=False):
    """
    Segmenta una imagen de página y procesa cada segmento.
    Incluye opciones para aplicar o no diferentes pasos de preprocesamiento.
    """
    try:
        img_original_pil = Image.open(imagen_path_original) # Abrir con PIL para obtener dimensiones
        ancho_total, altura_total = img_original_pil.size
    except Exception as e:
        print(f"Error abriendo la imagen original {imagen_path_original}: {e}")
        return []

    todos_los_datos_extraidos_pagina = []
    segmento_idx_global = 0

    # --- Estimaciones y Parámetros ---
    y_corte_horizontal_ratio = 0.49
    y_corte_horizontal = int(altura_total * y_corte_horizontal_ratio)

    secciones_tabla_y_coords = [
        (0, y_corte_horizontal),
        (y_corte_horizontal, altura_total)
    ]

    if y_corte_horizontal < int(altura_total * 0.2) or y_corte_horizontal > int(altura_total * 0.8):
        print(f"Página {os.path.basename(imagen_path_original)} parece tener una sola tabla principal o un corte no estándar. Procesando como una sola sección vertical.")
        secciones_tabla_y_coords = [(0, altura_total)]

    # Crear directorio para segmentos guardados si no existe
    segmentos_dir = "segmentos_guardados"
    if not os.path.exists(segmentos_dir):
        os.makedirs(segmentos_dir)
    nombre_base_original = os.path.splitext(os.path.basename(imagen_path_original))[0]

    for i, (y_start, y_end) in enumerate(secciones_tabla_y_coords):
        if y_end - y_start < 100:
            continue

        print(f"\nProcesando sección de tabla horizontal {i+1} (Y: {y_start}-{y_end}) de {os.path.basename(imagen_path_original)}")
        ancho_segmento_base = ancho_total // num_segmentos_verticales_por_tabla

        for j in range(num_segmentos_verticales_por_tabla):
            segmento_idx_global += 1
            x_segmento_start = j * ancho_segmento_base
            x_segmento_end = (j + 1) * ancho_segmento_base

            if j > 0: x_segmento_start -= overlap_px
            if j < num_segmentos_verticales_por_tabla - 1: x_segmento_end += overlap_px

            x_segmento_start = max(0, x_segmento_start)
            x_segmento_end = min(ancho_total, x_segmento_end)

            if x_segmento_start >= x_segmento_end:
                print(f"      Segmento {segmento_idx_global} inválido por superposición excesiva o tabla estrecha. Saltando.")
                continue

            print(f"    Procesando segmento vertical {j+1} de la sección {i+1} (Global ID: {segmento_idx_global}): X[{x_segmento_start}-{x_segmento_end}], Y[{y_start}-{y_end}]")

            segmento_pil = img_original_pil.crop((x_segmento_start, y_start, x_segmento_end, y_end))
            path_segmento_guardado = os.path.join(segmentos_dir, f"{nombre_base_original}_tabla{i+1}_seg{j+1}_orig.png")
            try:
                segmento_pil.save(path_segmento_guardado)
                print(f"      Segmento original guardado en: {path_segmento_guardado}")
            except Exception as e_save:
                print(f"      Error al guardar segmento original {path_segmento_guardado}: {e_save}")
                continue

            imagen_a_procesar_pil = segmento_pil
            identificador_imagen = path_segmento_guardado

            if aplicar_preprocesamiento:
                print(f"      Aplicando preprocesamiento al segmento...")
                try:
                    segmento_cv = cv2.cvtColor(np.array(segmento_pil), cv2.COLOR_RGB2BGR)
                    img_procesada_cv = segmento_cv.copy()

                    # --- INICIO CADENA DE PREPROCESAMIENTO (CONTROLADA POR FLAGS) ---
                    if aplicar_upscaling and (img_procesada_cv.shape[0] < 600 or img_procesada_cv.shape[1] < 800): # Condición de ejemplo
                        print(f"        - Aplicando upscaling (factor: {factor_upscaling})")
                        img_procesada_cv = cv2.resize(img_procesada_cv, None, fx=factor_upscaling, fy=factor_upscaling, interpolation=cv2.INTER_LANCZOS4)

                    if aplicar_escala_grises:
                        print("        - Convirtiendo a escala de grises")
                        img_procesada_cv = cv2.cvtColor(img_procesada_cv, cv2.COLOR_BGR2GRAY)
                        img_gray = img_procesada_cv # Mantener una referencia si se necesita

                        if aplicar_clahe:
                            print(f"        - Aplicando CLAHE (clipLimit={clip_limit_clahe}, tileGrid={tile_grid_clahe})")
                            clahe = cv2.createCLAHE(clipLimit=clip_limit_clahe, tileGridSize=tile_grid_clahe)
                            img_procesada_cv = clahe.apply(img_procesada_cv)

                        if aplicar_thresholding_adaptativo:
                             print(f"        - Aplicando thresholding adaptativo (blockSize={block_size_thresh}, C={c_value_thresh})")
                             img_procesada_cv = cv2.adaptiveThreshold(img_procesada_cv, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                                     cv2.THRESH_BINARY_INV, block_size_thresh, c_value_thresh) # Probando THRESH_BINARY_INV


                        if aplicar_denoising_mediano:
                            # Asegúrate de que kernel_size_denoising sea impar y > 1
                            if kernel_size_denoising % 2 == 0 or kernel_size_denoising <= 1:
                                print(f"        - Advertencia: kernel_size_denoising ({kernel_size_denoising}) debe ser impar y > 1. Usando 3.")
                                kernel_size_denoising = 3
                            print(f"        - Aplicando filtro mediano para eliminación de ruido (kernel={kernel_size_denoising})")
                            img_procesada_cv = cv2.medianBlur(img_procesada_cv, kernel_size_denoising)

                    if aplicar_nitidez:
                        print("        - Aplicando filtro de nitidez")
                        kernel_sharpen = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
                        img_procesada_cv = cv2.filter2D(img_procesada_cv, -1, kernel_sharpen)
                    # --- FIN CADENA DE PREPROCESAMIENTO ---

                    path_segmento_preproc = os.path.join(segmentos_dir, f"{nombre_base_original}_tabla{i+1}_seg{j+1}_preproc.png")
                    cv2.imwrite(path_segmento_preproc, img_procesada_cv)
                    print(f"      Segmento preprocesado guardado en: {path_segmento_preproc}")

                    # Convertir de nuevo a PIL Image para pasarlo a la función de extracción
                    # Detectar si la imagen es en escala de grises o color
                    if len(img_procesada_cv.shape) == 2 or img_procesada_cv.shape[2] == 1:
                        imagen_a_procesar_pil = Image.fromarray(img_procesada_cv, mode='L') # Escala de grises
                    else:
                         imagen_a_procesar_pil = Image.fromarray(cv2.cvtColor(img_procesada_cv, cv2.COLOR_BGR2RGB)) # Color

                    identificador_imagen = path_segmento_preproc

                except ImportError:
                    print("      OpenCV no está instalado. Saltando preprocesamiento.")
                except Exception as e_preproc:
                    print(f"      Error durante el preprocesamiento del segmento {path_segmento_guardado}: {e_preproc}. Usando segmento original.")
                    # Si falla el preprocesamiento, asegúrate de usar la imagen PIL original
                    imagen_a_procesar_pil = segmento_pil
                    identificador_imagen = path_segmento_guardado

            else:
                print("      Preprocesamiento desactivado para este segmento.")
                # Si el preprocesamiento general está desactivado, usar la imagen PIL original
                imagen_a_procesar_pil = segmento_pil
                identificador_imagen = path_segmento_guardado


            # Llamar a la función de extracción con la imagen (original o preprocesada)
            datos_segmento = extract_data_from_image_gemini_v_few_shot_pil(imagen_a_procesar_pil,
                                                                         target_dia_pago="31.12.2017",
                                                                         image_identifier=identificador_imagen)

            if datos_segmento:
                if isinstance(datos_segmento, list):
                    todos_los_datos_extraidos_pagina.extend(datos_segmento)
                else:
                    print(f"      Advertencia: Se esperaba una lista del segmento {identificador_imagen}, pero se obtuvo: {type(datos_segmento)}. Datos: {datos_segmento}")
            else:
                print(f"      No se extrajeron datos del segmento {identificador_imagen}.")

    return todos_los_datos_extraidos_pagina

In [ ]:
# CELDA NUEVA: Cargar y preparar la lista de meses/años a ignorar
import pandas as pd

ignore_csv_path = '/content/drive/MyDrive/DEUDA_PREVISIONAL/años_noconsiderar_2017/missing_year_months.csv' # Asegúrate que esta ruta sea correcta

try:
    df_ignore = pd.read_csv(ignore_csv_path)

    meses_map_num_to_nombre = {
        1: "ENERO", 2: "FEBRERO", 3: "MARZO", 4: "ABRIL", 5: "MAYO", 6: "JUNIO",
        7: "JULIO", 8: "AGOSTO", 9: "SEPTIEMBRE", 10: "OCTUBRE", 11: "NOVIEMBRE", 12: "DICIEMBRE"
    }

    def format_ignore_date(date_str_yyyy_mm):
        if pd.isna(date_str_yyyy_mm):
            return None
        try:
            year, month = map(int, date_str_yyyy_mm.split('-'))
            month_name = meses_map_num_to_nombre.get(month)
            if month_name:
                return f"{month_name} {year}"
            return None
        except Exception as e:
            print(f"Error formateando fecha a ignorar '{date_str_yyyy_mm}': {e}")
            return None

    if 'missing_year_month' in df_ignore.columns:
        df_ignore['formatted_ignore_date'] = df_ignore['missing_year_month'].apply(format_ignore_date)
        ignore_set = set(df_ignore['formatted_ignore_date'].dropna())
        print(f"Se cargarán {len(ignore_set)} meses/años para ignorar.")
        print("Primeros 5 meses/años a ignorar (formateados):", list(ignore_set)[:5])
    else:
        print(f"Error: La columna 'missing_year_month' no se encontró en {ignore_csv_path}")
        ignore_set = set()

except FileNotFoundError:
    print(f"Error: No se encontró el archivo CSV '{ignore_csv_path}'. No se ignorará ningún mes/año adicional.")
    ignore_set = set()
except Exception as e:
    print(f"Ocurrió un error al procesar el archivo CSV de ignorados: {e}")
    ignore_set = set()

Se cargarán 54 meses/años para ignorar.
Primeros 5 meses/años a ignorar (formateados): ['ENERO 2000', 'SEPTIEMBRE 2002', 'MAYO 2002', 'AGOSTO 2002', 'JUNIO 2011']


In [ ]:
# CELDA DEL BUCLE PRINCIPAL DE PROCESAMIENTO (MODIFICADA CON FILTRO Y OPCIÓN DE PREPROCESAMIENTO)

all_extracted_data_final = []
if not image_paths:
    print("No hay imágenes de página completa para procesar.")
else:
    # Procesar solo las primeras N páginas para pruebas rápidas, o todas con image_paths
    paginas_a_procesar = image_paths # <<< CAMBIA ESTO

    # Procesar solo las páginas con nombres específicos si están en image_paths
    # nombres_especificos = ['page_1.png', 'page_2.png']
    # paginas_a_procesar = [p for p in image_paths if os.path.basename(p) in nombres_especificos]

if not paginas_a_procesar:
    # print(f"Advertencia: No se encontraron las páginas '{nombres_especificos}' en la lista de imágenes encontradas.")
    if image_paths:
         print(f"Imágenes encontradas: {[os.path.basename(p) for p in image_paths]}")
    else:
         print("No se encontraron imágenes en la carpeta especificada.")
else:
    print(f"Se procesarán las siguientes páginas: {[os.path.basename(p) for p in paginas_a_procesar]}")

    for pagina_img_path in paginas_a_procesar:
        print(f"\n--- Procesando página completa: {pagina_img_path} ---")

        # --- CONFIGURACIÓN DEL PROCESAMIENTO DE LA PÁGINA Y PREPROCESAMIENTO ---
        num_segmentos = 2
        superposicion = 188
        aplicar_preproc_general = False # Flag general para activar/desactivar todo el preprocesamiento

        # Flags específicos para cada paso (solo se consideran si aplicar_preproc_general es True)
        usar_upscaling = False
        factor_upscale = 1.5
        usar_escala_grises = True # Generalmente recomendado para thresholding
        usar_clahe = True
        clip_limit_clahe_val = 2.0
        tile_grid_clahe_val = (8,8)
        usar_thresholding_adaptativo = False
        block_size_thresh_val = 11
        c_value_thresh_val = 2
        usar_denoising_mediano = False
        kernel_size_denoising_val = 3
        usar_nitidez = False # Usar con precaución

        print(f"Usando num_segmentos_verticales_por_tabla={num_segmentos}, overlap_px={superposicion}")
        print(f"Configuración de Preprocesamiento: General={aplicar_preproc_general}, Upscaling={usar_upscaling}, Gris={usar_escala_grises}, CLAHE={usar_clahe}, Thresholding={usar_thresholding_adaptativo}, Denoising={usar_denoising_mediano}, Nitidez={usar_nitidez}")


        datos_de_pagina_segmentada = segmentar_y_procesar_pagina_v2(
            pagina_img_path,
            num_segmentos_verticales_por_tabla=num_segmentos,
            overlap_px=superposicion,
            aplicar_preprocesamiento=aplicar_preproc_general, # Pasar el flag general
            aplicar_upscaling=usar_upscaling,
            factor_upscaling=factor_upscale,
            aplicar_escala_grises=usar_escala_grises,
            aplicar_clahe=usar_clahe,
            clip_limit_clahe=clip_limit_clahe_val,
            tile_grid_clahe=tile_grid_clahe_val,
            aplicar_thresholding_adaptativo=usar_thresholding_adaptativo,
            block_size_thresh=block_size_thresh_val,
            c_value_thresh=c_value_thresh_val,
            aplicar_denoising_mediano=usar_denoising_mediano,
            kernel_size_denoising=kernel_size_denoising_val,
            aplicar_nitidez=usar_nitidez
        )

        if datos_de_pagina_segmentada:
            all_extracted_data_final.extend(datos_de_pagina_segmentada)
        print(f"--- Fin del procesamiento para la página: {pagina_img_path} ---")


# ----- INICIO DE LA MODIFICACIÓN PARA FILTRAR -----
print(f"\nTotal de items extraídos antes de filtrar por CSV: {len(all_extracted_data_final)}")
if 'ignore_set' in globals() and ignore_set:
    all_extracted_data_temp_filtered = []
    for item_idx, item in enumerate(all_extracted_data_final):
        if isinstance(item, dict) and "mes_año_columna" in item:
            mes_ano_item_upper = str(item["mes_año_columna"]).upper()
            if mes_ano_item_upper.startswith("CIEMBRE "):
                mes_ano_item_upper = "DICIEMBRE " + mes_ano_item_upper.split(" ")[-1]
            if mes_ano_item_upper not in ignore_set:
                all_extracted_data_temp_filtered.append(item)
            else:
                print(f"  Item ignorado (de CSV): {item['mes_año_columna']} (comparado como {mes_ano_item_upper})")
        else:
            all_extracted_data_temp_filtered.append(item)
    all_extracted_data_final = all_extracted_data_temp_filtered
    print(f"Total de items después de filtrar por CSV: {len(all_extracted_data_final)}")
else:
    print("No se aplicó filtro por CSV (ignore_set no definido o vacío).")
# ----- FIN DE LA MODIFICACIÓN PARA FILTRAR -----

print("\n\n--- DATOS EXTRAÍDOS FINALES (segmentados y filtrados por CSV) ---")
if all_extracted_data_final:
    datos_unicos_temp = {}
    items_validos_para_deduplicacion = [item for item in all_extracted_data_final if isinstance(item, dict) and "mes_año_columna" in item]
    print(f"Total de items válidos para deduplicación: {len(items_validos_para_deduplicacion)}")

    for item_idx, item in enumerate(items_validos_para_deduplicacion):
        if isinstance(item, dict) and "mes_año_columna" in item:
            clave_unica_original = item["mes_año_columna"]
            clave_unica = str(clave_unica_original).upper()
            if clave_unica.startswith("CIEMBRE "):
                 clave_unica = "DICIEMBRE " + clave_unica.split(" ")[-1]

            def contar_nones(data_item):
                count = 0
                if not isinstance(data_item, dict): return 3
                if data_item.get("interes_afiliado_porcentaje") is None: count +=1
                if data_item.get("recargo_beneficio_afiliado_porcentaje") is None: count +=1
                if data_item.get("reajuste_porcentaje") is None: count +=1
                return count

            if clave_unica not in datos_unicos_temp:
                datos_unicos_temp[clave_unica] = item
            else:
                existente = datos_unicos_temp[clave_unica]
                nuevo = item
                nones_existente = contar_nones(existente)
                nones_nuevo = contar_nones(nuevo)
                if nones_nuevo < nones_existente:
                    print(f"  ACTUALIZANDO {clave_unica_original} (clave: {clave_unica}): Nueva versión tiene menos Nones ({nones_nuevo} vs {nones_existente}).")
                    datos_unicos_temp[clave_unica] = nuevo
    all_extracted_data_final_unicos = list(datos_unicos_temp.values())
    print(f"Total de items después de deduplicación: {len(all_extracted_data_final_unicos)}\n")

    def obtener_clave_ordenamiento(item):
        try:
            partes = item['mes_año_columna'].split()
            mes_str = partes[0]
            año_str = partes[1]
            meses_map = {"ENERO": 1, "FEBRERO": 2, "MARZO": 3, "ABRIL": 4, "MAYO": 5, "JUNIO": 6,
                         "JULIO": 7, "AGOSTO": 8, "SEPTIEMBRE": 9, "OCTUBRE": 10, "NOVIEMBRE": 11, "DICIEMBRE": 12,
                         "CIEMBRE": 12}
            mes_num = meses_map.get(mes_str.upper(), 0)
            return (int(año_str), mes_num)
        except:
            return (9999, 99)

    if all_extracted_data_final_unicos and isinstance(all_extracted_data_final_unicos[0], dict) and 'mes_año_columna' in all_extracted_data_final_unicos[0]:
       all_extracted_data_final_unicos.sort(key=obtener_clave_ordenamiento)
       print("  Datos reordenados cronológicamente.")

    for item in all_extracted_data_final_unicos:
        print(item)

    try:
        df_final = pd.DataFrame(all_extracted_data_final_unicos)
        print("\n--- DataFrame de Pandas (segmentado, filtrado y deduplicado) ---")
        print(df_final.to_string()) # Usar to_string() para ver más filas si es necesario

        # El guardado a Excel se moverá a la celda de comparación o alertas.
    except ImportError:
        print("\nPandas no está instalado. No se puede crear el DataFrame.")
    except Exception as e:
        print(f"Error al crear o mostrar el DataFrame final: {e}")
else:
    print("No se extrajeron datos o todos fueron filtrados/deduplicados (all_extracted_data_final_unicos está vacío).")
    df_final = pd.DataFrame() # Asegurarse que df_final exista como DataFrame vacío si no hay datos

Se procesarán las siguientes páginas: ['page_1.png', 'page_2.png', 'page_3.png', 'page_4.png', 'page_5.png', 'page_6.png', 'page_7.png', 'page_8.png', 'page_9.png', 'page_10.png', 'page_11.png', 'page_12.png', 'page_13.png', 'page_14.png', 'page_15.png', 'page_16.png', 'page_17.png', 'page_18.png', 'page_19.png', 'page_20.png', 'page_21.png', 'page_22.png', 'page_23.png', 'page_24.png', 'page_25.png', 'page_26.png', 'page_27.png', 'page_28.png', 'page_29.png', 'page_30.png', 'page_31.png']

--- Procesando página completa: /content/drive/MyDrive/DEUDA_PREVISIONAL/PDFS/2017/page_1.png ---
Usando num_segmentos_verticales_por_tabla=2, overlap_px=188
Configuración de Preprocesamiento: General=False, Upscaling=False, Gris=True, CLAHE=True, Thresholding=False, Denoising=False, Nitidez=False

Procesando sección de tabla horizontal 1 (Y: 0-1616) de page_1.png
    Procesando segmento vertical 1 de la sección 1 (Global ID: 1): X[0-1462], Y[0-1616]
      Segmento original guardado en: segmentos_gu

In [ ]:
# # Cargar el archivo CSV de datos reales
# csv_reales_path = '/content/drive/MyDrive/DEUDA_PREVISIONAL/datos_2017_reales.csv' # Asegúrate que esta ruta sea correcta

# try:
#     df_reales = pd.read_csv(csv_reales_path, sep=',') # Asegúrate que el separador sea el correcto
#     print(f"\nDataFrame de datos reales cargado desde '{csv_reales_path}':")
#     print(df_reales.head())
#     print(f"Columnas en df_reales: {df_reales.columns.tolist()}")

# except FileNotFoundError:
#     print(f"\nError: No se encontró el archivo CSV de datos reales en la ruta especificada: {csv_reales_path}")
#     df_reales = pd.DataFrame()
# except Exception as e:
#     print(f"\nOcurrió un error al cargar el archivo CSV de datos reales: {e}")
#     df_reales = pd.DataFrame()

# # Asegurarse de que df_final exista y no esté vacío antes de intentar la comparación
# if 'df_final' in locals() and not df_final.empty and not df_reales.empty:

#     rename_map = {
#         'reajuste_REAL_porcentaje': 'reajuste_porcentaje',
#         'interes_afiliado_REAL_porcentaje': 'interes_afiliado_porcentaje',
#         'recargo_beneficio_afiliado_REAL_porcentaje': 'recargo_beneficio_afiliado_porcentaje'
#     }
#     df_reales_renombrado = df_reales.rename(columns=rename_map)
#     print("\nDataFrame de datos reales renombrado para comparación:")
#     print(df_reales_renombrado.head())

#     if 'mes_año_columna' in df_final.columns and 'mes_año_columna' in df_reales_renombrado.columns:

#         # Normalizar 'mes_año_columna' para el merge (mayúsculas y sin espacios extra)
#         df_final['mes_año_columna_upper'] = df_final['mes_año_columna'].astype(str).str.upper().str.strip()
#         df_reales_renombrado['mes_año_columna_upper'] = df_reales_renombrado['mes_año_columna'].astype(str).str.upper().str.strip()

#         df_comparacion = pd.merge(
#             df_final,
#             df_reales_renombrado,
#             on='mes_año_columna_upper', # Usar la columna normalizada para el merge
#             suffixes=('_extraido', '_real'),
#             how='outer'
#         )

#         print("\nDataFrame de comparación (merge):")
#         print(df_comparacion.head().to_string())

#         def clean_and_convert_numeric(series_data):
#             if series_data is None:
#                 return pd.Series([pd.NA] * 1) # O la longitud que corresponda
#             # Convertir a string, luego limpiar y convertir a numérico
#             series_str = series_data.astype(str)
#             series_cleaned = series_str.str.replace('%', '', regex=False).str.replace(',', '.', regex=False)
#             return pd.to_numeric(series_cleaned, errors='coerce')


#         cols_to_compare = ['reajuste_porcentaje', 'interes_afiliado_porcentaje', 'recargo_beneficio_afiliado_porcentaje']
#         alertas = []

#         for col in cols_to_compare:
#             col_extraido = col + '_extraido'
#             col_real = col + '_real'

#             # Aplicar limpieza y conversión a las columnas del DataFrame de comparación
#             if col_extraido in df_comparacion.columns:
#                 df_comparacion[col_extraido + '_numeric'] = clean_and_convert_numeric(df_comparacion[col_extraido])
#             else:
#                  df_comparacion[col_extraido + '_numeric'] = pd.Series([pd.NA] * len(df_comparacion))

#             if col_real in df_comparacion.columns:
#                 df_comparacion[col_real + '_numeric'] = clean_and_convert_numeric(df_comparacion[col_real])
#             else:
#                 df_comparacion[col_real + '_numeric'] = pd.Series([pd.NA] * len(df_comparacion))


#             tolerancia = 1e-2

#             # Asegurarse que las columnas numéricas existen antes de operar
#             if (col_extraido + '_numeric' in df_comparacion.columns and
#                 col_real + '_numeric' in df_comparacion.columns):

#                 mask_ambos_existentes = df_comparacion[col_extraido + '_numeric'].notna() & df_comparacion[col_real + '_numeric'].notna()
#                 mask_diferentes = mask_ambos_existentes & (abs(df_comparacion[col_extraido + '_numeric'] - df_comparacion[col_real + '_numeric']) > tolerancia)

#                 for index, row in df_comparacion[mask_diferentes].iterrows():
#                     alerta = {
#                         'mes_año': row.get('mes_año_columna_upper', 'N/A'), # Usar .get para evitar KeyError
#                         'columna': col,
#                         'valor_extraido': row.get(col_extraido, 'N/A'),
#                         'valor_real': row.get(col_real, 'N/A'),
#                         'tipo_alerta': 'Diferencia significativa'
#                     }
#                     alertas.append(alerta)
#             else:
#                 print(f"Advertencia: Columnas numéricas para '{col}' no encontradas completamente en df_comparacion.")


#         # Meses/años extraídos pero no en los datos reales
#         # Usar mes_año_columna_extraido (de df_final) y mes_año_columna_real (de df_reales_renombrado)
#         # Estas columnas se crean por el merge si el on='mes_año_columna_upper' fue exitoso
#         # y las columnas originales eran 'mes_año_columna'
#         if 'mes_año_columna_extraido' in df_comparacion.columns and 'mes_año_columna_real' in df_comparacion.columns:
#             mask_solo_extraido = df_comparacion['mes_año_columna_real'].isna() & df_comparacion['mes_año_columna_extraido'].notna()
#             for index, row in df_comparacion[mask_solo_extraido].iterrows():
#                 alerta = {
#                     'mes_año': row.get('mes_año_columna_upper', 'N/A'),
#                     'columna': 'General',
#                     'valor_extraido': 'Presente',
#                     'valor_real': 'Ausente',
#                     'tipo_alerta': 'Mes/Año extraído, pero no en CSV real'
#                 }
#                 if not any(a['mes_año'] == alerta['mes_año'] and a['tipo_alerta'] == alerta['tipo_alerta'] for a in alertas):
#                     alertas.append(alerta)

#             mask_solo_real = df_comparacion['mes_año_columna_extraido'].isna() & df_comparacion['mes_año_columna_real'].notna()
#             for index, row in df_comparacion[mask_solo_real].iterrows():
#                 alerta = {
#                     'mes_año': row.get('mes_año_columna_upper', 'N/A'),
#                     'columna': 'General',
#                     'valor_extraido': 'Ausente',
#                     'valor_real': 'Presente',
#                     'tipo_alerta': 'Mes/Año en CSV real, pero no extraído'
#                 }
#                 if not any(a['mes_año'] == alerta['mes_año'] and a['tipo_alerta'] == alerta['tipo_alerta'] for a in alertas):
#                     alertas.append(alerta)
#         else:
#             print("Advertencia: Las columnas 'mes_año_columna_extraido' o 'mes_año_columna_real' no se encontraron después del merge. No se pueden generar alertas de solo_extraido/solo_real.")


#         print("\n--- ALERTA DE COMPARACIÓN ---")
#         if alertas:
#             for alerta in alertas:
#                 print(f"ALERTA: Mes/Año: {alerta['mes_año']}, Columna: {alerta['columna']}, Tipo: {alerta['tipo_alerta']}")
#                 if alerta['tipo_alerta'] == 'Diferencia significativa':
#                     print(f"  > Extraído: '{alerta['valor_extraido']}', Real: '{alerta['valor_real']}'")
#         else:
#             print("No se encontraron diferencias significativas ni meses/años faltantes en la comparación.")

#         if alertas:
#             df_alertas = pd.DataFrame(alertas)
#             print("\nDataFrame de Alertas:")
#             print(df_alertas.to_string())
#             excel_filename_final = f"datos_extraidos_comparados_alertas.xlsx"

#             try:
#                 with pd.ExcelWriter(excel_filename_final) as writer:
#                     if not df_final.empty:
#                         df_final.to_excel(writer, sheet_name='DatosExtraidos', index=False)
#                     if not df_comparacion.empty:
#                          df_comparacion.to_excel(writer, sheet_name='DataFrameComparacion', index=False)
#                     if not df_alertas.empty:
#                         df_alertas.to_excel(writer, sheet_name='AlertasComparacion', index=False)

#                 print(f"\nArchivos Excel guardados en '{excel_filename_final}'.")
#                 from google.colab import files
#                 files.download(excel_filename_final)
#                 print(f"Iniciando descarga de '{excel_filename_final}'...")
#             except Exception as e:
#                 print(f"Error al guardar el archivo Excel '{excel_filename_final}': {e}")
#         else:
#             if not df_final.empty:
#                 excel_filename_final_no_alertas = "datos_extraidos_final_sin_alertas_comp.xlsx"
#                 try:
#                     df_final.to_excel(excel_filename_final_no_alertas, index=False)
#                     print(f"\nDataFrame extraído (sin alertas de comparación) guardado como '{excel_filename_final_no_alertas}'.")
#                     from google.colab import files
#                     files.download(excel_filename_final_no_alertas)
#                     print(f"Iniciando descarga de '{excel_filename_final_no_alertas}'...")
#                 except Exception as e:
#                     print(f"Error al guardar el archivo Excel '{excel_filename_final_no_alertas}': {e}")
#     else:
#         print("\nLa columna 'mes_año_columna' no se encontró en uno o ambos DataFrames. No se puede realizar la comparación.")
# else:
#     print("\nDataFrame extraído (df_final) o DataFrame real (df_reales) no existe o está vacío. No se realizó la comparación.")
#     if 'df_final' in locals() and not df_final.empty:
#         excel_filename_solo_extraido = "datos_extraidos_final_solo.xlsx"
#         try:
#             df_final.to_excel(excel_filename_solo_extraido, index=False)
#             print(f"\nDataFrame extraído (sin comparación) guardado como '{excel_filename_solo_extraido}'.")
#             from google.colab import files
#             files.download(excel_filename_solo_extraido)
#             print(f"Iniciando descarga de '{excel_filename_solo_extraido}'...")
#         except Exception as e:
#             print(f"Error al guardar el archivo Excel '{excel_filename_solo_extraido}': {e}")

In [ ]:
if 'df_final' in locals() and not df_final.empty:
    def clean_percentage(perc_str):
        if pd.isna(perc_str) or perc_str is None:
            return None
        try:
            return float(str(perc_str).replace('%', '').replace(',', '.'))
        except ValueError:
            # Intenta manejar casos como "658.76" sin el %
            try:
                return float(str(perc_str).replace(',', '.'))
            except ValueError:
                return None


    df_final['interes_afiliado_porcentaje_clean'] = df_final['interes_afiliado_porcentaje'].apply(clean_percentage)
    df_final['recargo_beneficio_afiliado_porcentaje_clean'] = df_final['recargo_beneficio_afiliado_porcentaje'].apply(clean_percentage)

    # Asegurar que las columnas _clean sean numéricas para .diff()
    df_final['interes_afiliado_porcentaje_clean'] = pd.to_numeric(df_final['interes_afiliado_porcentaje_clean'], errors='coerce')
    df_final['recargo_beneficio_afiliado_porcentaje_clean'] = pd.to_numeric(df_final['recargo_beneficio_afiliado_porcentaje_clean'], errors='coerce')


    df_final['variacion_interes'] = df_final['interes_afiliado_porcentaje_clean'].diff().abs()
    df_final['alerta_interes'] = (df_final['variacion_interes'] > 100) & (df_final['interes_afiliado_porcentaje_clean'].shift(1).notna())

    df_final['variacion_recargo'] = df_final['recargo_beneficio_afiliado_porcentaje_clean'].diff().abs()
    df_final['alerta_recargo'] = (df_final['variacion_recargo'] > 20) & (df_final['recargo_beneficio_afiliado_porcentaje_clean'].shift(1).notna())

    df_final['alerta_general_variacion'] = df_final['alerta_interes'] | df_final['alerta_recargo']

    print("\n--- DataFrame con Columnas de Alerta de Variación Interna ---")
    cols_to_show_alertas_internas = ['mes_año_columna',
                                     'interes_afiliado_porcentaje', 'interes_afiliado_porcentaje_clean', 'variacion_interes', 'alerta_interes',
                                     'recargo_beneficio_afiliado_porcentaje', 'recargo_beneficio_afiliado_porcentaje_clean', 'variacion_recargo', 'alerta_recargo',
                                     'alerta_general_variacion']
    # Filtrar para mostrar solo las columnas que existen en df_final
    cols_to_show_alertas_internas_existentes = [col for col in cols_to_show_alertas_internas if col in df_final.columns]
    print(df_final[cols_to_show_alertas_internas_existentes].to_string())


    if not df_final.empty:
        excel_filename_alertas_variacion = "datos_extraidos_final_CON_ALERTAS_VARIACION.xlsx"
        try:
            # Si la celda anterior (comparación) ya guardó un excel, podríamos querer añadir estas alertas como una nueva hoja
            # o sobreescribir/crear uno nuevo. Aquí se crea/sobrescribe.
            with pd.ExcelWriter(excel_filename_alertas_variacion) as writer:
                df_final.to_excel(writer, sheet_name='DatosConAlertasVariacion', index=False)
                # Si df_comparacion y df_alertas (de la celda anterior) existen y quieres incluirlos:
                if 'df_comparacion' in locals() and not df_comparacion.empty:
                    df_comparacion.to_excel(writer, sheet_name='ComparacionConReales', index=False)
                if 'df_alertas' in locals() and isinstance(df_alertas, pd.DataFrame) and not df_alertas.empty:
                    df_alertas.to_excel(writer, sheet_name='AlertasComparacionReales', index=False)

            print(f"\nDataFrame con alertas de variación guardado como '{excel_filename_alertas_variacion}'.")
            from google.colab import files
            files.download(excel_filename_alertas_variacion)
            print(f"Iniciando descarga de '{excel_filename_alertas_variacion}'...")
        except Exception as e:
            print(f"Error al guardar el archivo Excel '{excel_filename_alertas_variacion}': {e}")
    else:
        print("\nEl DataFrame final está vacío, no se guardará en Excel con alertas de variación.")
else:
    print("\nEl DataFrame 'df_final' no existe o está vacío. No se pueden generar alertas de variación interna.")


--- DataFrame con Columnas de Alerta de Variación Interna ---
     mes_año_columna interes_afiliado_porcentaje  interes_afiliado_porcentaje_clean  variacion_interes  alerta_interes recargo_beneficio_afiliado_porcentaje  recargo_beneficio_afiliado_porcentaje_clean  variacion_recargo  alerta_recargo  alerta_general_variacion
0          MAYO 1997                    3293.80%                            3293.80                NaN           False                               658.76%                                       658.76                NaN           False                     False
1         JUNIO 1997                    3250.14%                            3250.14              43.66           False                               650.03%                                       650.03               8.73           False                     False
2         JULIO 1997                    3207.96%                            3207.96              42.18           False                              

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Iniciando descarga de 'datos_extraidos_final_CON_ALERTAS_VARIACION.xlsx'...


In [ ]:
# # Esta celda asume que la Celda 5 se ha ejecutado
# # y que la variable 'df_final' existe y es un DataFrame de Pandas.

# # Guardar a Excel (COMENTADO)
# if not df_final.empty:
#     excel_filename_final = "datos_extraidos_segmentados_final.xlsx"
#     df_final.to_excel(excel_filename_final, index=False)
#     print(f"\nDataFrame segmentado guardado como '{excel_filename_final}'.")

#     from google.colab import files # COMENTADO
#     files.download(excel_filename_final) # COMENTADO
# else:
#     print("\nEl DataFrame final está vacío, no se guardará en Excel.")